# Sample notebook for codeblocks

Run `experimental.ipynb` to get the files out

## Loading Data

In [2]:
import os
import sys

# Add parent folder to sys.path
sys.path.append(os.path.abspath(".."))
from functools import reduce

import math
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from helpers.load_data import load_and_merge_sensors
from helpers.filtering import preprocess_sensor_data
from helpers.segment import segment_series
import helpers.dtwlabeling

import seglearn as seg
import tensorflow as tf
import sklearn

/Users/hammy/Projects/CS156/Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
training = pd.read_pickle('../data/training.pkl')
test = pd.read_pickle('../data/test.pkl')

In [10]:
X = training['X_seg']
y = training['y_exercise']
subject = training['subject_windows']

array([[[-2.03323861e-03, -4.23621531e-03,  1.69245417e-02, ...,
          4.18854157e-02,  6.73056555e-03, -1.46123169e-02],
        [ 2.95196642e-01, -5.66283493e-02, -5.07909759e-02, ...,
         -5.43837827e-02,  1.89266861e-01,  5.63682990e-03],
        [ 4.93771362e-01, -3.84294067e-02, -1.10721891e-01, ...,
         -1.14844097e-01,  3.06297232e-01,  1.90224735e-02],
        ...,
        [ 8.42459828e-03, -2.96978047e-01,  1.27609034e-01, ...,
         -5.46923078e-02,  6.07259806e-02,  1.58466276e-02],
        [ 1.05183882e-01, -3.16910500e-01,  3.85722219e-02, ...,
         -7.25734662e-02,  8.28002549e-02,  1.12834883e-02],
        [ 1.80750779e-01, -2.49011163e-01, -6.28761629e-02, ...,
         -1.08699451e-01,  9.21416165e-02,  1.91541777e-02]],

       [[ 8.84604319e-02,  1.88319646e-01, -1.29800349e-01, ...,
         -6.98826951e-02, -4.26961633e-02,  6.21791427e-02],
        [ 8.22816801e-02,  5.13560436e-02,  4.14498436e-03, ...,
         -5.46593913e-02, -5.33274723e

## Sample Training

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Flatten, Dense, Dropout, BatchNormalization

def build_model(window_size, num_sensors, num_classes, use_lstm=False):
    """
    Builds a Keras model for multivariate time-series sensor data.

    Parameters:
    - window_size: int, number of timesteps per window
    - num_sensors: int, number of sensor channels
    - num_classes: int, number of output classes
    - use_lstm: bool, if True adds an LSTM layer after CNN

    Returns:
    - Keras compiled model
    """
    model = Sequential()
    
    # First CNN layer
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(window_size, num_sensors)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # Second CNN layer
    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    if use_lstm:
        model.add(LSTM(64, return_sequences=False))
    else:
        model.add(Flatten())
    
    # Dense layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [21]:
from tensorflow.keras.utils import to_categorical
import numpy as np

subjects = ['s1', 's2', 's3', 's4', 's5']
num_classes = 8  # 8

loso_accuracies = []

for test_subject in np.unique(subjects):
    train_idx = subject != test_subject
    test_idx = subject == test_subject

    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]

    # One-hot encode labels
    y_train_onehot = to_categorical(y_train - 1, num_classes=num_classes)
    y_test_onehot = to_categorical(y_test - 1, num_classes=num_classes)

    # Build your model
    model = build_model(window_size=X_train.shape[1], num_sensors=X_train.shape[2], num_classes=num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train
    model.fit(X_train, y_train_onehot, epochs=20, batch_size=32, verbose=0)

    # Evaluate
    loss, acc = model.evaluate(X_test, y_test_onehot, verbose=0)
    print(f"Test subject {test_subject}: accuracy = {acc:.3f}")
    loso_accuracies.append(acc)

print(f"Average LOSO accuracy: {np.mean(loso_accuracies):.3f}")

/Users/hammy/Projects/CS156/Project/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test subject s1: accuracy = 0.552
Test subject s2: accuracy = 0.944
Test subject s3: accuracy = 0.859
Test subject s4: accuracy = 0.941
Test subject s5: accuracy = 0.962
Average LOSO accuracy: 0.852
